In [2]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.1 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 9.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 68.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-9d9dpkc2/overlay/local/bin
    changing mode of /tmp/pip-build-env-9d9dpkc2/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-9d9dpkc2/overlay/lo

In [3]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [4]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [5]:

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:

from google.colab import files


pd= files.upload()


Saving tagged_data_llama_256_1.csv to tagged_data_llama_256_1.csv


In [8]:
import pandas as pd
import random
df = pd.read_csv('tagged_data_llama_256_1.csv')


In [9]:
df = df[['reviews', 'Correct review ', 'Incorrect review ']]
df.head()

,reviews,Correct review,Incorrect review
0,Not bad but could be better,1,0
1,Taking biotin 5000mcg from October first week ...,1,0
2,My nails grew in noticeably thicker and strong...,1,0
3,I took biotin for 1 week. I started having mu...,1,0
4,I used to have very thin nails that bent when ...,1,0


In [10]:
df_incorrect = df.loc[df['Incorrect review '] == 1]['reviews'].reset_index(drop=True)
df_correct = df.loc[df['Correct review '] == 1]['reviews'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

"I have been on 5000/day for a little over a month now and have really not seen any difference but I've been told it takes 1-2 months so I'm hoping at 38 Biotin helps me grow back the thinning hair on top and I can finally wear bangs again."

In [11]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'good'

In [ ]:
#Step 6: Generating the Response

In [12]:
for _ in range(20):
    prompt = f'''Examples of drug review ratings:
    1. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    2. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    3. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1
    4. {df_incorrect[random.choice(range(len(df_incorrect)))][:256]}
     ANSWER: 0
    5. {df_correct[random.choice(range(len(df_correct)))][:256]}
     ANSWER: 1

   Respond in the json format.You are a product review evaluator. Your task is to assess the correctness of only following review for dietary supplements and provide a label (1 for correct, 0 for incorrect):
    "{df['reviews'][random.choice(range(len(df)))]}"
    '''
    prompt_template=f'''SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.

    USER: {prompt}

    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)
    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    matches = re.findall(r'ASSISTANT:.*?([01])', response_text, re.DOTALL)

    print(matches)



{'id': 'cmpl-de75aada-346a-4b5f-8ad6-403ba28358c8', 'object': 'text_completion', 'created': 1701634804, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. I am taking 5000 mcg of Biotin per day.  Gradually over 3 mo. my finger nails are strengthening, although they still have ridges and crack on occasion.\n     ANSWER: 1\n    2. Take Part of my morning regimen. Prescribed/recommended by my family physician.\n     ANSWER: 0\n    3. sleep beter\n     ANSWER: 1\n    4. Vitamin D3 - take 2000 a day.\n     ANSWER: 0\n    5. I take this about 30 minutes before I\'m ready to go to sleep. Helps me relax and makes me sleepy so I fall asleep easily.\n    

Llama.generate: prefix-match hit


{'id': 'cmpl-7b959610-a3c6-4464-ad73-f9c913b1a3aa', 'object': 'text_completion', 'created': 1701634846, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. Just the smell and taste isn\'t good esp if they leek as they do some times\n     ANSWER: 1\n    2. Test in QA for count!\n     ANSWER: 0\n    3. Somewhat effective.  Has an after wake up feeling of unfresh\n     ANSWER: 1\n    4. IT IS OKAY\n     ANSWER: 0\n    5. I took biotin 5000Mg it helped hair growth but it cause me to GAIN WEIGHT. I excersize regularly but i noticed i got a bit heavier\n     ANSWER: 1\n\n   Respond in the json format.You are a product review evaluator. Your task is to 

Llama.generate: prefix-match hit


{'id': 'cmpl-e1db07b8-7e78-4d6a-9139-2d3b14972ea4', 'object': 'text_completion', 'created': 1701634896, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. I used its tablets 06 months, I feel my mind works better.\n     ANSWER: 1\n    2. has any9ne noticed a bad taste I mouth whentaking Vt D?\n     ANSWER: 0\n    3. Had the flu and used Colloidal Silver the gargle in mouth one and it worked wonders. I was not able to keep food down and with the first dose, the next morning and I eating and feeling 100% better.\n     ANSWER: 1\n    4. BOLD FACE LYING MEDICAL INDUSTRY\n     ANSWER: 0\n    5. I developed an extremely itchy scalp.  The itch and rash

...

{'id': 'cmpl-f31528f3-946c-4e01-bd27-70f479d271ab', 'object': 'text_completion', 'created': 1701635267, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. ive had prostate infections for over 20 years. usually it takes 30 days of some serious cipro to cure. i had a bad infection, couldn\'t get antibiotics, took the silver and the infection was completely gone within 5 days.\n     ANSWER: 1\n    2. Too soo to determine effectiveness, I had an A1c of 6.1, but I have a low iron which could present a false high. Are we supposed to be on this forever or just until the tests are normal?\n     ANSWER: 0\n    3. I took very high dose for about 28 days t

Llama.generate: prefix-match hit


{'id': 'cmpl-275a3ec3-92fa-473a-bbc9-b59b7d4d0ae1', 'object': 'text_completion', 'created': 1701635288, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. hair was falling out so bad that hairdresser was having a difficult time doing my hair, this has helped 100%\n     ANSWER: 1\n    2. Only true CS reduced and capped NOT IONIC silver which is not CS\n     ANSWER: 0\n    3. Decreased back pain (have rods and screws in back due to burst fracture of a vertebrae) and increased mental clarity.\n     ANSWER: 1\n    4. This article says it discusses the side effects of vitamin D.  It discusses none of the side effects, merely the intended effects of t

Llama.generate: prefix-match hit


{'id': 'cmpl-d8e02953-af79-4d22-b59e-4e3df0230c79', 'object': 'text_completion', 'created': 1701635311, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a helpful, respectful, and honest assistant. Your task is to accurately evaluate drug reviews for correctness based on the provided examples.\n\n    USER: Examples of drug review ratings:\n    1. I take 50,000 units once per week for low vitamin D readings on my last blood check up. I am having headaches that seem to be coming more frequently and the pain is getting worse. This is the end of my second week of a 6 week treatment.\n     ANSWER: 1\n    2. good stuff!!!!!!!!\n     ANSWER: 0\n    3. Was not compatible with IBS! Didn\'t have a good experience at all.\n     ANSWER: 1\n    4. I wish to now somthig about D50000,Dr.give me the treatmen for 4 week.\n     ANSWER: 0\n    5. Has ass

In [13]:
!pip install scikit-learn

true_labels = [1,0,1,1,1,1,1,0,1,1]
predicted_labels = [1,1,0,0,1,1,1,1,1,1,]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 60.00%
